[comment]: <> (The book says: "Decorator attaches additional responsibilities to an object dynamically. Provides a flexible alternative to subclassing for extending functionality.")

The Decorator pattern lets you extend the functionality of a class without modifying the class' code and without subclassing.

## Our first design

Say we're building an ordering system for a coffee shop, and we want to design classes to represent the drinks ordered. The system will create an object for each drink, and will need to have each object return its description and price.

Our first design uses a Beverage class that knows about the condiments that can be included in a drink (e.g. milk, mocha). Subclasses of Beverage represent the different types of coffee (e.g. House Blend, Decaf) have the drink-specific descriptions and prices.

In [1]:
class Beverage:
    milkCost = 0.10
    soyCost = 0.15
    mochaCost = 0.20
    
    def __init__(self, hasMilk=False, hasSoy=False, hasMocha=False):
        self.hasMilk = hasMilk
        self.hasSoy = hasSoy
        self.hasMocha = hasMocha
        
    def __str__(self):
        return "Unknown Beverage"
    
    def get_condiment_cost(self):
        condimentCost = 0.0
        if self.hasMilk:
            condimentCost += self.milkCost
        if self.hasSoy:
            condimentCost += self.soyCost
        if self.hasMocha:
            condimentCost += self.mochaCost
        return condimentCost
    
    def get_condiment_desc(self):
        condimentDesc = ""
        if self.hasMilk:
            condimentDesc += ", add Milk"
        if self.hasSoy:
            condimentDesc += ", add Soy"
        if self.hasMocha:
            condimentDesc += ", add Mocha"
        return condimentDesc
    
###########################
class HouseBlend(Beverage):
    def __str__(self):
        return "House Blend" + super().get_condiment_desc()
    
    def get_cost(self):
        return 0.89 + super().get_condiment_cost() 
    
class DarkRoast(Beverage):
    def __str__(self):
        return "Dark Roast" + super().get_condiment_desc()
    
    def get_cost(self):
        return 0.99 + super().get_condiment_cost()
    
class Decaf(Beverage):    
    def __str__(self):
        return "Decaf" + super().get_condiment_desc()
    
    def get_cost(self):
        return 1.05 + super().get_condiment_cost()

Let's see how to use these classes.

A list of drink objects represents the order, and we can have the objects print out their descriptions and prices.

In [2]:
order = [Decaf(hasMilk=True, hasMocha=True),
         DarkRoast(hasSoy=True)]

for bev in order:
    print(f'{bev}: ${bev.get_cost()}')

Decaf, add Milk, add Mocha: $1.35
Dark Roast, add Soy: $1.14


### The burden of this design

Now suppose our coffee shop adds a new condiment to the menu: Whip. Because knowledge of the condiments was designed into the Beverage class, that's where we need to make changes to add this menu item.

In [3]:
class Beverage:
    milkCost = 0.10
    soyCost = 0.15
    mochaCost = 0.20
    whipCost = 0.10                         # <--- add a class variable # <1>
    
    def __init__(self, hasMilk=False, hasSoy=False, hasMocha=False, 
                 hasWhip=False):            # <--- change the interface # <2>
        self.hasMilk = hasMilk              
        self.hasSoy = hasSoy
        self.hasMocha = hasMocha
        self.hasWhip = hasWhip              # <--- add an instance variable # <3>
        
    def __str__(self):
        return "Unknown Beverage"
    
    def get_condiment_cost(self):
        condimentCost = 0.0
        if self.hasMilk:
            condimentCost += self.milkCost
        if self.hasSoy:
            condimentCost += self.soyCost
        if self.hasMocha:
            condimentCost += self.mochaCost
        if self.hasWhip:                    # <--- add new logic # <4>
            condimentCost += self.whipCost
        return condimentCost
    
    def get_condiment_desc(self):
        condimentDesc = ""
        if self.hasMilk:
            condimentDesc += ", add Milk"
        if self.hasSoy:
            condimentDesc += ", add Soy"
        if self.hasMocha:
            condimentDesc += ", add Mocha"
        if self.hasWhip:                    # <--- add new logic # <5>
            condimentDesc += ", add Whip"
        return condimentDesc

In [4]:
#| echo: False

# These don't change, but we do need to re-run this block of code

class HouseBlend(Beverage):
    def __str__(self):
        return "House Blend" + super().get_condiment_desc()
    
    def get_cost(self):
        return 0.89 + super().get_condiment_cost() 
    
class DarkRoast(Beverage):
    def __str__(self):
        return "Dark Roast" + super().get_condiment_desc()
    
    def get_cost(self):
        return 0.99 + super().get_condiment_cost()
    
class Decaf(Beverage):    
    def __str__(self):
        return "Decaf" + super().get_condiment_desc()
    
    def get_cost(self):
        return 1.05 + super().get_condiment_cost()

Now we can add Whip to our drinks.

In [5]:
order = [Decaf(hasMilk=True, hasMocha=True),
         DarkRoast(hasWhip=True)]

for bev in order:
    print(f'{bev}: ${bev.get_cost()}')

Decaf, add Milk, add Mocha: $1.35
Dark Roast, add Whip: $1.09


Notice the big disadvantage of this design. To add the new condiment, we were forced to make changes to almost every part of the Beverage class. But Beverage was already working well; we would really prefer to leave it alone rather than open it up and risk breaking something.

## An easy-to-change design using Decorator

[comment]: <> (Next: look at the various versions side-by-side, for easier comparison)

Let's see how the Decorator pattern makes our design easier to change.

In [6]:
# Beverage and its subclasses know nothing about condiments:

class Beverage:       
    def __str__(self):
        return "Unknown Beverage"


class HouseBlend(Beverage):
    def get_cost(self):
        return 0.89
    
    def __str__(self):
        return "House Blend"
        
class DarkRoast(Beverage):
    def get_cost(self):
        return 0.99
    
    def __str__(self):
        return "Dark Roast"
    
class Decaf(Beverage):
    def get_cost(self):
        return 1.05
    
    def __str__(self):
        return "Decaf"
    
############################
# Here are the decorators

class CondimentDecorator(Beverage):
    def __init__(self, beverage):
        super().__init__()
        self.beverage = beverage
        
class Milk(CondimentDecorator):
    def __str__(self):
        return self.beverage.__str__() + ", add Milk"
    
    def get_cost(self):
        return self.beverage.get_cost() + 0.10

class Soy(CondimentDecorator):
    def __str__(self):
        return self.beverage.__str__() + ", add Soy"
    
    def get_cost(self):
        return self.beverage.get_cost() + 0.15
    
class Mocha(CondimentDecorator):
    def __str__(self):
        return self.beverage.__str__() + ", add Mocha"
    
    def get_cost(self):
        return self.beverage.get_cost() + 0.20

The usage of these classes now looks different. Instead of specifying the condiments in the call to the constructor, we wrap the constructor with the decorators:

In [7]:
order = [Milk(Mocha(Decaf())),
         Soy(DarkRoast())]

for bev in order:
    print(f'{bev}: ${bev.get_cost()}')

Decaf, add Mocha, add Milk: $1.35
Dark Roast, add Soy: $1.14


### The payoff

When we enhance our system with the new Whip condiment, we don't need to modify *any* existing class. We only have to define a new subclass of the decorator. 

In [8]:
class Whip(CondimentDecorator):
    def __str__(self):
        return self.beverage.__str__() + ", add Whip"
    
    def get_cost(self):
        return self.beverage.get_cost() + 0.10

And with that one addition, we can add Whip to the menu:

In [9]:
order = [Milk(Mocha(Decaf())),
         Whip(DarkRoast())]

for bev in order:
    print(f'{bev}: ${bev.get_cost()}')

Decaf, add Mocha, add Milk: $1.35
Dark Roast, add Whip: $1.09
